# Code to create KLD df and Scrambled/Null KL DF

In [2]:
#not sure if this is important
#import sys
#sys.path.append('/mnt/ibm_lg/sarah.ancheta')
#import Inter_IntraDF as zf

In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl


from scipy.special import rel_entr, kl_div
from scipy.special import softmax
from scipy.stats import entropy
from scipy import stats

from functools import reduce

In [4]:
from joblib import Parallel, delayed

# Interfish analysis (KL)

In [5]:
# function: calculation of KL for two numpy arrays
# arguments: two numpy arrays
# output: KL score (float)

def kl_divergence(fish1gene = [],  fish2gene = []):
    
    
    pkde = stats.gaussian_kde(np.array(fish1gene))
    qkde = stats.gaussian_kde(np.array(fish2gene))
    maxval = np.max([np.max(fish1gene), np.max(fish2gene)])
    minval = np.min([np.min(fish1gene), np.min(fish2gene)])
    
    ### check gaussian_kde, no parameter that could change results (ie. range)
    offset = 1
    newx = np.linspace(minval-offset, maxval+offset,1000)
    
    p = pkde.pdf(newx)
    q = qkde.pdf(newx)
    
    newp = np.where(p< 1e-9, 1e-9, p)
    newq = np.where(q< 1e-9, 1e-9, q)

    
    kl_pair = stats.entropy(newp, newq)
    return kl_pair

Function gets the arrays of RNA counts for the gene/cell type

In [6]:
# function: helper to make the matrix, gets the specific arrays and sends to kl function
# arguments: matrix of counts (sparse), gene, cell type, two fish (str), layer (log/scaled str), obs and var from adata as pd.DF
# output: kl divergence score for two arrays (calls kl function)

def kl_fishpair_genecelltype2(matrix = [[],[]], gene = 'phtf2', cluster = 'Somite', fish1 = '44', fish2 = '46', layer = 'log', obs=pd.DataFrame, var=pd.DataFrame):
       
 
    row1 = obs['fish'].loc[lambda x: x==fish1].index.tolist()
    row2 = obs['fish'].loc[lambda x: x==fish2].index.tolist()
    
    myindex = int(var.index[var['index']==gene].tolist()[0])
    
    if (layer == 'scaled'):
        q = np.asarray(matrix[row1, myindex]).flatten()
        p = np.asarray(matrix[row2, myindex]).flatten()
    
    if (layer == 'log'):
        q = np.asarray(matrix.tocsr()[row1, myindex].todense()).flatten()
        p = np.asarray(matrix.tocsr()[row2, myindex].todense()).flatten()
    
    ##have to fix, not sure how
    if (layer == 'counts'):
        mymatrix = myadata.layers.copy()
    
    if (np.sum(p > 0) < 10) or (np.sum(q > 0) < 10):
        return -1
    else:
        return kl_divergence(q, p)

Makes a matrix of pairwise fish

In [7]:
# function: makes a matrix with the pairwise KL values
# arguments: takes in a matrix of counts (sparse), a gene, cell type, list of fish (str), layer (log/scaled str), obs and var from adata as pd.DF
# output: a matrix with all pairwise KL scores for the fish

def kl_fish_matrix2(matrix = [[],[]], gene = 'phtf2', cluster = 'Somite', fish = [], layer = 'log', obs =pd.DataFrame, var=pd.DataFrame):

    fishcols = []
    
    for f1 in fish:
        f1row = []
        for f2 in fish:
            value = kl_fishpair_genecelltype2(matrix, gene, cluster, f1, f2, layer, obs, var)
            
            if (value == -1):
                return -1
            f1row.append(value)
        fishcols.append(f1row)      

    return fishcols 

Summarizes the pairwise fish matrix (excluding the diagonal elements, compared to self) by the numpy method specified

In [8]:
# function: calculates summary metric of kl matrix pairwise scores, not including the diagonal elements
# arguments: matrix with kl pairwise scores, summary method metric (numpy)
# output: single summary number of matrix (float)

def kl_matrix_summary3(myfishmatrix, method = np.max):
    testmatrix = np.array(myfishmatrix)
    numzeros = len(testmatrix)
    newx = np.sort(testmatrix, axis=None)
    
    allelem = newx[numzeros:]

    

    return method(allelem)

Function that does everything, pretty much, but we have a helper so fewer vars have to be specified.

In [46]:
# function: parallelized version of making dataframe with KL metrics, per gene, grouped by cell type
# arguments: adata, list of genes, one cell type, out_path as str, title as string 
# output: returns a dataframe with KL metrics per gene, per cell type

def KL_metrics_genecluster_parallel(adata = [], gene = [], cluster = 'Somite', myfish = [], layer = 'scaled', method = [], timepoint = 'hpf'):
    
    myadata = adata[adata.obs['Cells_Annotations'] == cluster].copy()
    a_var = myadata.var.copy().reset_index()
    a_obs = myadata.obs.copy().reset_index()
    
    
    
    if (layer == 'scaled'):
        newmatrix = myadata.X.copy()
    
    if (layer == 'log'):
        newmatrix = myadata.X.copy()
        #newmatrix = myadata.raw.X.copy()
    
    ##have to fix, not sure how
    if (layer == 'counts'):
        newmatrix = myadata.layers.copy()
    
    
    
    mygenes = Parallel(n_jobs=-1)(delayed(kl_fish_matrix2)(newmatrix, g, cluster, myfish, layer, a_obs, a_var) for g in gene)
    mygenes = [{'mymatrix':g} for g in mygenes if g != -1]
    for generow in mygenes:
        for m in method:
            stat = kl_matrix_summary3(generow['mymatrix'], m)
            generow[m.__name__] = stat
    bigframe = pd.DataFrame.from_dict(mygenes)
    bigframe['timepoint'] = timepoint
    
    return bigframe


In [48]:
# function: return a dataframe with the desired KL metric summaries for specified genes, for a certain cell type
# arguments: adata object, list of genes, cell type (str), list of fish (str), layer (log/scaled str), list of np summary methods (mean/median, etc)
# output: pd dataframe, with columns gene, cluster (cell type), and summary methods


def KL_metrics_genecluster(adata = [], gene = [], cluster = 'Somite', myfish = [], layer = 'scaled', method = []):
    
    myadata = adata[adata.obs['Cells_Annotations'] == cluster].copy()
    a_var = myadata.var.copy().reset_index()
    a_obs = myadata.obs.copy().reset_index()
    
    
    
    if (layer == 'scaled'):
        newmatrix = myadata.X.copy()
    
    if (layer == 'log'):
        newmatrix = myadata.X.copy()
        #newmatrix = myadata.raw.X.copy()
    
    ##have to fix, not sure how
    if (layer == 'counts'):
        newmatrix = myadata.layers.copy()
    
    
    mygenes = []
    generow = {}
    for g in gene:
        mymatrix = kl_fish_matrix2(newmatrix, g, cluster, myfish, layer, a_obs, a_var)
        
        if mymatrix == -1:
            continue
        
        generow = {'gene': g, 'cluster': cluster}
        for m in method:
            stat = kl_matrix_summary3(mymatrix, m)
            
            generow[m.__name__] = stat

        mygenes.append(generow)
    
        
    bigframe = pd.DataFrame.from_dict(mygenes)
    return bigframe

This function computes the KL dataframe.

In [10]:
# function: creates a dataframe with KL metrics, per gene, grouped by cell type
# arguments: adata, list of cell types/annotations, out_path as str, title as string 
# output: returns a dataframe with KL metrics per gene, per cell type

def KLdf(adata = [], annotations = ['Hindbrain'], myfish = [], timepoint = 'hpf'):
    KLframes = []
    mymetrics = [np.max, np.min, np.median, np.std, np.mean]
    mygenes = np.asarray(adata.var_names)
    for a in annotations:
        miniframe = KL_metrics_genecluster(adata, mygenes, a, myfish, 'log', mymetrics, timepoint)
        KLframes.append(miniframe)


    KL_metrics = pd.concat(KLframes, axis=0)
    
    return KL_metrics

### Creates and saves the KL dataframe

#### This is the function we call. We just input the adata object and the cell types we want to run it for.

In [11]:
# function: exports a csv dataframe with KL metrics, per gene, grouped by cell type
# arguments: adata, list of cell types/annotations, out_path as str, title as string 
# output: saves a CSV dataframe with KL metrics per gene, per cell type at the out_path

def saveKLdf(adata = [], annotations = ['Hindbrain'], out_path = '', save_title = 'date', timepoint = 'hpf'):
    myfish = adata.obs['fish'].unique()
    KL_metrics = KLdf(adata, annotations, myfish, timepoint)
    

    KL_metrics.to_csv(out_path + save_title + 'Inter.csv')
    
    return KL_metrics


## Creates scrambled fish frame

In [12]:
# function: saves a csv pandas DF with intrafish metrics, with numrandomstates different random scrambled fish states
# arguments: adata, cell types list of str, out_path (str) and title to save (str)
# output: csv with scrambled fish, same as intrafish DF but with added column of the random state

def scrambledInterFish(adata = [], annotations = ['Hindbrain'], numrandomstates = 20):
    
    scrambleDFframes = []
    myfish = adata.obs['fish'].unique()
    
    for i in range(numrandomstates):
        adata1 = adata.copy()
        adata1.obs['fish'] = np.random.RandomState(seed=i).permutation(adata1.obs['fish'].values)
        state = str(i)
        
        fishscramble = KLdf(adata1, annotations, myfish)
        fishscramble['Random_State']='mixed_fish' + state
        scrambleDFframes.append(fishscramble)
        
        
    scrambleInter = pd.concat(scrambleDFframes, axis=0)
    
    
    return scrambleInter


In [13]:
# function: saves scrambled (null) df with KL analysis
# arguments: adata, list of cell types/annotations, number of randomzied iterations, out_path as str, title as string 
# output: returns a dataframe with KL metrics per gene, per cell type, with random state column (which iteration of random)

def save_scrambledInter(adata = [], annotations = ['Hindbrain'], numrandomstates = 20, out_path = '', save_title = 'date'):
    
    mydf = scrambledInterFish(adata, annotations, numrandomstates)
    mydf.to_csv(out_path + save_title + 'null_scramble.csv')
    
    return mydf

## Upload adata and get KLD dataframe for a single timepoint

### helper functions to subset/prepare input

In [14]:
# function: subsets adata for a timepoint
# arguments: adata object and desired timepoint
# output: subset adata object
    
def myadata(adata, timepoint):
    myadata = adata[adata.obs.timepoint == timepoint].copy()
    
    return myadata

In [15]:
# function: some preprocessing, go from adata object to input for 
# input: adata object, and either a list of cell types with the cell type column name, or 'cell_type_1' if whole embryo
# output: adata with new 'Cells_Annotations' column, the cell type(s) of interest for analysis

def make_columns(adata, annotations = ['cell_type_1'], celltypecolumn = 'ontology_class'):
    
    if annotations[0] == 'cell_type_1':
        newadata = adata.copy()
        newadata.obs['Cells_Annotations'] = 'cell_type_1'
        
    else:
        newadata = adata[adata.obs[celltypecolumn].isin(annotations)].copy()
        newadata.obs['Cells_Annotations'] = newadata.obs[celltypecolumn].copy()
        
    
    return newadata


## Whole embryo KLD df

In [16]:
wholeadata = sc.read_h5ad('/mnt/ibm_lg/alejandro/danio-atlas/zebrahub/final_objects/v4/zf_atlas_full_v4_release.h5ad')

In [17]:
adata12hpf = myadata(wholeadata, '12hpf')

In [18]:
adata12hpf.obs.zebrafish_anatomy_ontology_class.value_counts()

central_nervous_system    3035
paraxial_mesoderm         1378
periderm                   967
lateral_mesoderm           779
notochord                  185
mesenchyme                 135
endoderm                   100
intermediate_mesoderm       93
hematopoietic_system        19
neural_crest                 8
Name: zebrafish_anatomy_ontology_class, dtype: int64

In [37]:
test_adata = make_columns(adata12hpf, ['notochord'], 'zebrafish_anatomy_ontology_class')

In [20]:
test_adata.obs.fish.unique()

['TDR36', 'TDR38', 'TDR37', 'TDR35']
Categories (4, object): ['TDR35', 'TDR36', 'TDR37', 'TDR38']

In [21]:
%%time

KLdf(test_adata, ['notochord'], ['TDR36', 'TDR38', 'TDR37', 'TDR35'], '12hpf')

CPU times: user 1h 4min 22s, sys: 14min 19s, total: 1h 18min 41s
Wall time: 1h 19min 5s


,mymatrix,amax,amin,median,std,mean,timepoint
0,"[[0.0, 0.3598603311805409, 0.19441748756107174...",2.553160,0.064801,0.310346,0.716122,0.614513,12hpf
1,"[[0.0, 0.039580632392869464, 0.131123486964690...",2.488071,0.039581,0.255808,0.641599,0.481106,12hpf
2,"[[0.0, 0.1258755150082693, 0.09380573484883221...",0.164907,0.017569,0.099905,0.041140,0.093790,12hpf
3,"[[0.0, 0.11325393040308719, 0.0828979727099173...",0.337559,0.038942,0.087662,0.075830,0.109769,12hpf
4,"[[0.0, 0.25260132429756815, 0.683571640405637,...",2.082565,0.179502,0.689981,0.558820,0.808473,12hpf
...,...,...,...,...,...,...,...
5830,"[[0.0, 0.6063895011151754, 0.9509917088366951,...",0.978971,0.081270,0.511403,0.299165,0.521987,12hpf
5831,"[[0.0, 0.18193575151231522, 0.1335190169128627...",0.391585,0.039035,0.125186,0.105327,0.164634,12hpf
5832,"[[0.0, 0.42588165255345367, 0.0449298793499717...",0.705431,0.030001,0.178694,0.196004,0.246404,12hpf
5833,"[[0.0, 0.2807048279479995, 0.5918228308889479,...",0.690780,0.075390,0.208108,0.202605,0.274026,12hpf


In [45]:
test_adata.obs

,n_genes,n_counts,fish,timepoint_cluster,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_nc,pct_counts_nc,developmental_stage,timepoint,zebrafish_anatomy_ontology_class,zebrafish_anatomy_ontology_id,Cells_Annotations
cell_id,,,,,,,,,,,,,,,
TDR36_TTACCGCAGCGTGCCT-1,6105,46712.0,TDR36,13,6105,46712.0,1492.0,3.194040,1583.0,3.388851,05 somites,12hpf,notochord,ZFA:0000135,notochord
TDR36_TGATGGTCATACAGGG-1,3581,17960.0,TDR36,13,3581,17960.0,455.0,2.533407,710.0,3.953229,05 somites,12hpf,notochord,ZFA:0000135,notochord
TDR38_ACCGTTCGTCAGACGA-1,4946,25498.0,TDR38,13,4946,25498.0,631.0,2.474704,834.0,3.270845,05 somites,12hpf,notochord,ZFA:0000135,notochord
TDR38_CTTTCGGTCACCGGGT-1,3806,18105.0,TDR38,13,3806,18105.0,321.0,1.772991,542.0,2.993648,05 somites,12hpf,notochord,ZFA:0000135,notochord
TDR38_GCACGGTGTAAGTAGT-1,5271,34802.0,TDR38,13,5271,34802.0,667.0,1.916556,590.0,1.695305,05 somites,12hpf,notochord,ZFA:0000135,notochord
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TDR37_TCATCATTCTTTCCGG-1,4240,19232.0,TDR37,13,4240,19232.0,349.0,1.814684,331.0,1.721090,05 somites,12hpf,notochord,ZFA:0000135,notochord
TDR36_CAGGCCAGTGGTTTGT-1,5659,41339.0,TDR36,13,5659,41339.0,1295.0,3.132635,1290.0,3.120540,05 somites,12hpf,notochord,ZFA:0000135,notochord
TDR36_CCGATGGCAGCGCTTG-1,4304,22387.0,TDR36,13,4304,22387.0,656.0,2.930272,774.0,3.457364,05 somites,12hpf,notochord,ZFA:0000135,notochord


In [42]:
out_path = '/mnt/ibm_lg/sarah.ancheta/ZF_atlas/intra_inter_v4/'
save_title_embryo = 'whole_embryo12hpf_Aug30_'
save_title_celltype = 'cell_type12hpf_Aug30_'

In [37]:
whole_embryo12hpf = make_columns(adata12hpf, ['cell_type_1'])

In [57]:
%%time

saveIntraDf(whole_embryo12hpf, ['cell_type_1'], out_path, save_title_embryo, '12hpf')

CPU times: user 8h 58min 35s, sys: 47min 37s, total: 9h 46min 13s
Wall time: 9h 46min 44s


,gene,cluster,amax,amin,median,std,mean
0,ptpn12,central_nervous_system,0.364841,0.019112,0.087735,0.093928,0.123494
1,phtf2-1,central_nervous_system,0.252489,0.027272,0.092024,0.062370,0.110294
2,si:zfos-932h1.3,central_nervous_system,0.159848,0.017062,0.086439,0.042165,0.088226
3,lrp6,central_nervous_system,0.354018,0.035298,0.091897,0.100948,0.129532
4,dusp16,central_nervous_system,0.157430,0.033855,0.071790,0.037738,0.083361
...,...,...,...,...,...,...,...
10115,NC_002333.16,lateral_mesoderm,0.824243,0.020321,0.243166,0.249095,0.298514
10116,NC_002333.15,lateral_mesoderm,0.718458,0.062548,0.136598,0.171295,0.225634
10117,mt-nd5,lateral_mesoderm,0.306344,0.031528,0.113050,0.075731,0.146078
10118,mt-nd6,lateral_mesoderm,0.976742,0.041779,0.261635,0.321757,0.384380


## Scrambled null distribution for whole embryo

In [24]:
out_path_null = '/mnt/ibm_lg/sarah.ancheta/ZF_atlas/intra_inter_v4/'
save_title_null = 'whole_embryo_12hpf_Dec2'

In [25]:
%%time

save_scrambledInter(whole_embryo12hpf, ['cell_type_1'], 20, out_path_null, save_title_null)

CPU times: user 4d 13h 44min 44s, sys: 10h 55min 8s, total: 5d 39min 52s
Wall time: 5d 1h 16min 27s


,gene,cluster,amax,amin,median,std,mean,Random_State
0,ptpn12,cell_type_1,0.083626,0.006492,0.039092,0.023252,0.041090,mixed_fish0
1,phtf2-1,cell_type_1,0.043425,0.005300,0.016523,0.011565,0.018588,mixed_fish0
2,si:zfos-932h1.3,cell_type_1,0.035981,0.004834,0.013897,0.011322,0.017569,mixed_fish0
3,lrp6,cell_type_1,0.096752,0.007861,0.034588,0.026393,0.039353,mixed_fish0
4,dusp16,cell_type_1,0.021946,0.006107,0.009837,0.004973,0.011127,mixed_fish0
...,...,...,...,...,...,...,...,...
16392,mt-nd5,cell_type_1,0.034979,0.006170,0.009854,0.008300,0.015054,mixed_fish19
16393,mt-nd6,cell_type_1,0.014722,0.002344,0.007687,0.003767,0.008980,mixed_fish19
16394,mt-cyb,cell_type_1,0.027284,0.006618,0.010755,0.005674,0.012853,mixed_fish19
16395,NC-002333.22,cell_type_1,0.071382,0.009398,0.026214,0.017339,0.033157,mixed_fish19
